In [1]:
from pynq import MMIO, Overlay
from pynq.lib import AxiGPIO
import time
import numbers

In [2]:
overlay = Overlay('/home/xilinx/pynq/overlays/EBAZ4205_4_AXIQuadSPI.overlay/EBAZ4205_4_AXIQuadSPI.bit')

In [ ]:
overlay?

- Block Design implemented in `EBAZ4205_4_AXIQuadSPI.bit`,<br><br>
![](../../resource/EBAZ4205_4_AXIQuadSPI_BlockDesign.png)<br><br><br><br>
- SPI pin is assigned to `A20` (MISO), `H16` (MOSI), `B19` (SCK), `B20` (CS) in `DATA1 PORT`, 
- Also LCD related GPIO via AxiGPIO, `C20` (LCD DC), `H17` (LCD Backlight), `D20` (LCD RESET)
![](../../resource/EBAZ4205_4_AXIQuadSPI_Pin.png)

In [3]:
SPI_regSpace = overlay.ip_dict['axi_quad_spi_0']

In [4]:
IP_BASE_ADDRESS = SPI_regSpace['phys_addr']
IP_ADDRESS_RNGE = SPI_regSpace['addr_range']
AxiQspi = MMIO(IP_BASE_ADDRESS, IP_ADDRESS_RNGE)

In [5]:
gpio_instance = overlay.ip_dict["axi_gpio_0"]
gpio = AxiGPIO(gpio_instance).channel1

In [6]:
# Xilinx Quad SPI IP Documentation :
# https://docs.amd.com/r/en-US/pg153-axi-quad-spi/XIP-Mode?tocId=2XVwkWB5eiUa4e0XIQSNuA

XSP_DGIER_OFFSET = 0x1C
XSP_IISR_OFFSET = 0x20
XSP_IIER_OFFSET = 0x28
XSP_SRR_OFFSET = 0x40
XSP_CR_OFFSET = 0x60
XSP_SR_OFFSET = 0x64
XSP_DTR_OFFSET = 0x68
XSP_DRR_OFFSET = 0x6C
XSP_SSR_OFFSET = 0x70
XSP_TFO_OFFSET = 0x74
XSP_RFO_OFFSET = 0x78
XSP_REGISTERS = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x00000004
XSP_SR_TX_FULL_MASK	= 0x00000008
XSP_CR_TRANS_INHIBIT_MASK = 0x00000100
XSP_CR_LOOPBACK_MASK	= 0x00000001
XSP_CR_ENABLE_MASK	= 0x00000002
XSP_CR_MASTER_MODE_MASK	= 0x00000004
XSP_CR_CLK_POLARITY_MASK = 0x00000008
XSP_CR_CLK_PHASE_MASK	= 0x00000010
XSP_CR_TXFIFO_RESET_MASK = 0x00000020
XSP_CR_RXFIFO_RESET_MASK = 0x00000040
XSP_CR_MANUAL_SS_MASK	= 0x00000080
XSP_CR_TRANS_INHIBIT_MASK = 0x00000100

SLAVE_NO_SELECTION = 0xFFFFFFFF

In [ ]:
def cnfg(AxiQspi, clk_phase=0, clk_pol=0):
	#print("Configure device")
	AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
	AxiQspi.write(XSP_DGIER_OFFSET, 0)
	AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
	ControlReg = AxiQspi.read(XSP_CR_OFFSET)
	ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK
	AxiQspi.write(XSP_CR_OFFSET, ControlReg)
	ControlReg = AxiQspi.read(XSP_CR_OFFSET)
	if clk_phase == 1:
		ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
	else:
		ControlReg = ControlReg & ~XSP_CR_CLK_PHASE_MASK
	if clk_pol == 1:
		ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
	else:
		ControlReg = ControlReg & ~XSP_CR_CLK_POLARITY_MASK
	AxiQspi.write(XSP_CR_OFFSET, ControlReg)

	return 0

def xfer(packet, AxiQspi):
	#print("TransferData")
	for data in packet:
		AxiQspi.write(XSP_DTR_OFFSET, data)
		AxiQspi.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
		ControlReg = AxiQspi.read(XSP_CR_OFFSET)
		ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK
		AxiQspi.write(XSP_CR_OFFSET, ControlReg)
		
		StatusReg = AxiQspi.read(XSP_SR_OFFSET)
		while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
			StatusReg = AxiQspi.read(XSP_SR_OFFSET)

		#print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(AxiQspi.read(XSP_RFO_OFFSET)))
		ControlReg = AxiQspi.read(XSP_CR_OFFSET)
		ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK
		AxiQspi.write(XSP_CR_OFFSET, ControlReg)

	AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)


- Test SPI transfer data

In [ ]:
cnfg(AxiQspi)

In [ ]:
xfer([0xFF, 0xF0, 0x00, 0x01], AxiQspi)

____________________________

# ST7735 SPI TFT LCD Driver Demo
- Source : https://github.com/pimoroni/st7735-python

In [7]:
# Constants for interacting with display registers.

BG_SPI_CS_BACK = 0
BG_SPI_CS_FRONT = 1

ST7735_TFTWIDTH = 130
ST7735_TFTHEIGHT = 130

ST7735_COLS = 130
ST7735_ROWS = 130

ST7735_NOP = 0x00
ST7735_SWRESET = 0x01
ST7735_RDDID = 0x04
ST7735_RDDST = 0x09

ST7735_SLPIN = 0x10
ST7735_SLPOUT = 0x11
ST7735_PTLON = 0x12
ST7735_NORON = 0x13

ST7735_INVOFF = 0x20
ST7735_INVON = 0x21
ST7735_DISPOFF = 0x28
ST7735_DISPON = 0x29

ST7735_CASET = 0x2A
ST7735_RASET = 0x2B
ST7735_RAMWR = 0x2C
ST7735_RAMRD = 0x2E

ST7735_PTLAR = 0x30
ST7735_MADCTL = 0x36
ST7735_COLMOD = 0x3A

ST7735_FRMCTR1 = 0xB1
ST7735_FRMCTR2 = 0xB2
ST7735_FRMCTR3 = 0xB3
ST7735_INVCTR = 0xB4
ST7735_DISSET5 = 0xB6


ST7735_PWCTR1 = 0xC0
ST7735_PWCTR2 = 0xC1
ST7735_PWCTR3 = 0xC2
ST7735_PWCTR4 = 0xC3
ST7735_PWCTR5 = 0xC4
ST7735_VMCTR1 = 0xC5

ST7735_RDID1 = 0xDA
ST7735_RDID2 = 0xDB
ST7735_RDID3 = 0xDC
ST7735_RDID4 = 0xDD

ST7735_GMCTRP1 = 0xE0
ST7735_GMCTRN1 = 0xE1

ST7735_PWCTR6 = 0xFC

# Colours for convenience
ST7735_BLACK = 0x0000  # 0b 00000 000000 00000
ST7735_BLUE = 0x001F  # 0b 00000 000000 11111
ST7735_GREEN = 0x07E0  # 0b 00000 111111 00000
ST7735_RED = 0xF800  # 0b 11111 000000 00000
ST7735_CYAN = 0x07FF  # 0b 00000 111111 11111
ST7735_MAGENTA = 0xF81F  # 0b 11111 000000 11111
ST7735_YELLOW = 0xFFE0  # 0b 11111 111111 00000
ST7735_WHITE = 0xFFFF  # 0b 11111 111111 11111

In [8]:
# utility functions

import numpy as np

def image_to_data(image, rotation=0):
    """Generator function to convert a PIL image to 16-bit 565 RGB bytes."""
    # NumPy is much faster at doing this. NumPy code provided by:
    # Keith (https://www.blogger.com/profile/02555547344016007163)
    pb = None
    if isinstance(image, Image.Image):
        pb = np.array(image).astype('uint16')
    elif isinstance(image, np.ndarray):
        pb = image.astype('uint16')
    else:
        raise ValueError("Unsupported image type!")
    
    # Handle rotation if required
    if rotation in [90, 180, 270]:
        k = rotation // 90  # Number of 90-degree rotations
        pb = np.rot90(pb, k=k)

    # Conver to RGB565
    color = ((pb[:, :, 0] & 0xF8) << 8) | ((pb[:, :, 1] & 0xFC) << 3) | (pb[:, :, 2] >> 3) 

    return np.dstack(((color >> 8) & 0xFF, color & 0xFF)).flatten().tolist()

In [9]:
class ST7735(object):
    """Representation of an ST7735 TFT LCD."""

    def __init__(self, spi, gpio, dc=0, bl=1, rst=2, width=ST7735_TFTWIDTH,
                 height=ST7735_TFTHEIGHT, rotation=0, offset_left=None, offset_top=None, invert=False, bgr=True):
        """Create an instance of the display using SPI communication.

        Must provide the GPIO pin label for the D/C pin and the SPI driver.

        Can optionally provide the GPIO pin label for the reset pin as the rst parameter.

        :param backlight: Pin for controlling backlight
        :param rst: Reset pin for ST7735
        :param width: Width of display connected to ST7735
        :param height: Height of display connected to ST7735
        :param rotation: Rotation of display connected to ST7735
        :param offset_left: COL offset in ST7735 memory
        :param offset_top: ROW offset in ST7735 memory
        :param invert: Invert display

        """

        self._spi = spi
        self._spi_cnfg(self._spi)
        self._spi_control_reg = self._spi.read(XSP_CR_OFFSET)
        self._spi_fifo_depth = 256  # Set FIFO depth

        self._gpio = gpio
        self._dc = dc

        self._width = width
        self._height = height
        self._rotation = rotation
        self._invert = invert
        self._bgr = bgr

        self._bl = bl
        self._rst = rst

        # Default left offset to center display
        if offset_left is None:
            offset_left = (ST7735_COLS - width) // 2

        self._offset_left = offset_left

        # Default top offset to center display
        if offset_top is None:
            offset_top = (ST7735_ROWS - height) // 2

        self._offset_top = offset_top
        
        # Setup Backlight to turn on
        if bl is not None:
            self.set_pin(self._bl, True)

        self.reset()
        self._init()

    def _spi_cnfg(self, AxiQspi, clk_phase=0, clk_pol=0):
        AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
        AxiQspi.write(XSP_DGIER_OFFSET, 0)
        AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
        ControlReg = AxiQspi.read(XSP_CR_OFFSET)
        ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)
        ControlReg = AxiQspi.read(XSP_CR_OFFSET)
        if clk_phase == 1:
            ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
        else:
            ControlReg = ControlReg & ~XSP_CR_CLK_PHASE_MASK
        if clk_pol == 1:
            ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
        else:
            ControlReg = ControlReg & ~XSP_CR_CLK_POLARITY_MASK
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)

        return 0
    
    def _spi_xfer(self, packet, AxiQspi):
        """Optimized SPI transfer function utilizing FIFO."""
        packet_len = len(packet)
        packet_index = 0

        # Configure Control Register once at the beginning
        ControlReg = self._spi_control_reg #AxiQspi.read(XSP_CR_OFFSET)
        ControlReg &= ~XSP_CR_TRANS_INHIBIT_MASK  # Enable transfer
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)
        
        # Enable the slave select
        AxiQspi.write(XSP_SSR_OFFSET, 0xFFFFFFFE)

        # Batch-write to the FIFO in chunks
        while packet_index < packet_len:
            # Determine how much data to write in the current batch
            batch_size = min(self._spi_fifo_depth, packet_len - packet_index)

            # Write data to the FIFO
            for i in range(batch_size):
                AxiQspi.write(XSP_DTR_OFFSET, packet[packet_index + i])
            packet_index += batch_size

            # Wait for the FIFO to empty before proceeding to the next batch
            # while (AxiQspi.read(XSP_SR_OFFSET) & XSP_SR_TX_EMPTY_MASK) == 0:
            #     pass

        # Disable the slave select after the transfer
        AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)

        # Reconfigure Control Register to inhibit transfer
        ControlReg = self._spi_control_reg #AxiQspi.read(XSP_CR_OFFSET)
        ControlReg |= XSP_CR_TRANS_INHIBIT_MASK  # Disable transfer
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    def set_pin(self, pin, state):
        self._gpio[pin].write(state)
    
    def send(self, data, is_data=True, chunk_size=4096):
        """Write a byte or array of bytes to the display. Is_data parameter
        controls if byte should be interpreted as display data (True) or command
        data (False).  Chunk_size is an optional size of bytes to write in a
        single SPI transaction, with a default of 4096.
        """
        # Set DC low for command, high for data.
        self.set_pin(self._dc, is_data)
        # Convert scalar argument to list so either can be passed as parameter.
        if isinstance(data, numbers.Number):
            data = [data & 0xFF]
        self._spi_xfer(data, self._spi)

    def set_backlight(self, value):
        """Set the backlight on/off."""
        if self._bl is not None:
            self.set_pin(self._bl, value)

    def display_off(self):
        self.command(ST7735_DISPOFF)

    def display_on(self):
        self.command(ST7735_DISPON)

    def sleep(self):
        self.command(ST7735_SLPIN)

    def wake(self):
        self.command(ST7735_SLPOUT)

    @property
    def width(self):
        return self._width if self._rotation == 0 or self._rotation == 180 else self._height

    @property
    def height(self):
        return self._height if self._rotation == 0 or self._rotation == 180 else self._width

    def command(self, data):
        """Write a byte or array of bytes to the display as command data."""
        self.send(data, False)

    def data(self, data):
        """Write a byte or array of bytes to the display as display data."""
        self.send(data, True)

    def reset(self):
        """Reset the display, if reset pin is connected."""
        if self._rst is not None:
            self.set_pin(self._rst, True)
            time.sleep(0.500)
            self.set_pin(self._rst, False)
            time.sleep(0.500)
            self.set_pin(self._rst, True)
            time.sleep(0.500)

    def _init(self):
        # Initialize the display.

        self.command(ST7735_SWRESET)    # Software reset
        time.sleep(0.150)               # delay 150 ms

        self.command(ST7735_SLPOUT)     # Out of sleep mode
        time.sleep(0.500)               # delay 500 ms

        self.command(ST7735_FRMCTR1)    # Frame rate ctrl - normal mode
        self.data(0x01)                 # Rate = fosc/(1x2+40) * (LINE+2C+2D)
        self.data(0x2C)
        self.data(0x2D)

        self.command(ST7735_FRMCTR2)    # Frame rate ctrl - idle mode
        self.data(0x01)                 # Rate = fosc/(1x2+40) * (LINE+2C+2D)
        self.data(0x2C)
        self.data(0x2D)

        self.command(ST7735_FRMCTR3)    # Frame rate ctrl - partial mode
        self.data(0x01)                 # Dot inversion mode
        self.data(0x2C)
        self.data(0x2D)
        self.data(0x01)                 # Line inversion mode
        self.data(0x2C)
        self.data(0x2D)

        self.command(ST7735_INVCTR)     # Display inversion ctrl
        self.data(0x07)                 # No inversion

        self.command(ST7735_PWCTR1)     # Power control
        self.data(0xA2)
        self.data(0x02)                 # -4.6V
        self.data(0x84)                 # auto mode

        self.command(ST7735_PWCTR2)     # Power control
        self.data(0x0A)                 # Opamp current small
        self.data(0x00)                 # Boost frequency

        self.command(ST7735_PWCTR4)     # Power control
        self.data(0x8A)                 # BCLK/2, Opamp current small & Medium low
        self.data(0x2A)

        self.command(ST7735_PWCTR5)     # Power control
        self.data(0x8A)
        self.data(0xEE)

        self.command(ST7735_VMCTR1)     # Power control
        self.data(0x0E)

        if self._invert:
            self.command(ST7735_INVON)   # Invert display
        else:
            self.command(ST7735_INVOFF)  # Don't invert display

        self.command(ST7735_MADCTL)     # Memory access control (directions)
        if self._bgr:
            self.data(0xC8)             # row addr/col addr, bottom to top refresh; Set D3 RGB Bit to 1 for format BGR
        else:
            self.data(0xC0)             # row addr/col addr, bottom to top refresh; Set D3 RGB Bit to 0 for format RGB

        self.command(ST7735_COLMOD)     # set color mode
        self.data(0x05)                 # 16-bit color

        self.command(ST7735_CASET)      # Column addr set
        self.data(0x00)                 # XSTART = 0
        self.data(self._offset_left)
        self.data(0x00)                 # XEND = ROWS - height
        self.data(self._width + self._offset_left - 1)

        self.command(ST7735_RASET)      # Row addr set
        self.data(0x00)                 # XSTART = 0
        self.data(self._offset_top)
        self.data(0x00)                 # XEND = COLS - width
        self.data(self._height + self._offset_top - 1)

        self.command(ST7735_GMCTRP1)    # Set Gamma
        self.data(0x02)
        self.data(0x1c)
        self.data(0x07)
        self.data(0x12)
        self.data(0x37)
        self.data(0x32)
        self.data(0x29)
        self.data(0x2d)
        self.data(0x29)
        self.data(0x25)
        self.data(0x2B)
        self.data(0x39)
        self.data(0x00)
        self.data(0x01)
        self.data(0x03)
        self.data(0x10)

        self.command(ST7735_GMCTRN1)    # Set Gamma
        self.data(0x03)
        self.data(0x1d)
        self.data(0x07)
        self.data(0x06)
        self.data(0x2E)
        self.data(0x2C)
        self.data(0x29)
        self.data(0x2D)
        self.data(0x2E)
        self.data(0x2E)
        self.data(0x37)
        self.data(0x3F)
        self.data(0x00)
        self.data(0x00)
        self.data(0x02)
        self.data(0x10)

        self.command(ST7735_NORON)      # Normal display on
        time.sleep(0.10)                # 10 ms

        self.display_on()
        time.sleep(0.100)               # 100 ms

    def begin(self):
        """Set up the display

        Deprecated. Included in __init__.

        """
        pass

    def set_window(self, x0=0, y0=0, x1=None, y1=None):
        """Set the pixel address window for proceeding drawing commands. x0 and
        x1 should define the minimum and maximum x pixel bounds.  y0 and y1
        should define the minimum and maximum y pixel bound.  If no parameters
        are specified the default will be to update the entire display from 0,0
        to width-1,height-1.
        """
        if x1 is None:
            x1 = self._width - 1

        if y1 is None:
            y1 = self._height - 1

        y0 += self._offset_top
        y1 += self._offset_top

        x0 += self._offset_left
        x1 += self._offset_left

        self.command(ST7735_CASET)       # Column addr set
        self.data(x0 >> 8)
        self.data(x0)                    # XSTART
        self.data(x1 >> 8)
        self.data(x1)                    # XEND
        self.command(ST7735_RASET)       # Row addr set
        self.data(y0 >> 8)
        self.data(y0)                    # YSTART
        self.data(y1 >> 8)
        self.data(y1)                    # YEND
        self.command(ST7735_RAMWR)       # write to RAM

    def display(self, image):
        """Write the provided image to the hardware.

        :param image: Should be RGB format and the same dimensions as the display hardware.

        """
        # Set address bounds to entire display.
        self.set_window()
        # Convert image to array of 16bit 565 RGB data bytes.
        # Unfortunate that this copy has to occur, but the SPI byte writing
        # function needs to take an array of bytes and PIL doesn't natively
        # store images in 16-bit 565 RGB format.
        pixelbytes = image_to_data(image, self._rotation)
        # Write data to hardware.
        self.data(pixelbytes)

In [10]:
from PIL import Image, ImageOps
from IPython.display import display

img1 = Image.open("pynq.jpg")
img2 = Image.open("xilinx.jpg")

In [ ]:
display(img1)

In [13]:
# example display image
disp = ST7735(               
   AxiQspi, gpio, dc=0, bl=1, rst=2
)
WIDTH = disp.width
HEIGHT = disp.height

# Initialize display.
disp.begin()

# Resize the image
img1_resized = img1.resize((WIDTH, HEIGHT))
img2_resized = img2.resize((WIDTH, HEIGHT))

# convert image to numpy array
img1_arr = np.array(img1_resized)
img2_arr = np.array(img2_resized)

print("start displaying images...")
for i in range(20):

    start_time = time.time()
    disp.display(img1_arr)
    elapsed_time_img1 = time.time() - start_time
    print(f"Time to display img1: {elapsed_time_img1:.4f} seconds")

#     start_time = time.time()
#     disp.display(img2_arr)
#     elapsed_time_img2 = time.time() - start_time
#     print(f"Time to display img2: {elapsed_time_img2:.4f} seconds")

start displaying images...
Time to display img1: 0.9211 seconds
Time to display img1: 0.9148 seconds
Time to display img1: 0.9190 seconds
Time to display img1: 0.9095 seconds
Time to display img1: 0.9176 seconds
Time to display img1: 0.9150 seconds
Time to display img1: 0.9182 seconds
Time to display img1: 0.9175 seconds
Time to display img1: 0.9196 seconds
Time to display img1: 0.9162 seconds
Time to display img1: 0.9217 seconds
Time to display img1: 0.9147 seconds
Time to display img1: 0.9133 seconds
Time to display img1: 0.9166 seconds
Time to display img1: 0.9210 seconds
Time to display img1: 0.9154 seconds
Time to display img1: 0.9184 seconds
Time to display img1: 0.9190 seconds
Time to display img1: 0.9184 seconds
Time to display img1: 0.9166 seconds


- Experiment result draw image to SPI LCD TFT ST7735
![](../../resource/EBAZ4205_4_AXIQuadSPI_Photo.jpg)